In [1]:
"""
install：
curl -L -O https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-MacOSX-arm64.sh
bash Miniforge3-MacOSX-arm64.sh

conda create -n cnn-mac python=3.12
conda activate cnn-mac
pip install tensorflow-macos tensorflow-metal numpy pandas opencv-python scikit-image scikit-learn
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
"""

'\ninstall：\ncurl -L -O https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-MacOSX-arm64.sh\nbash Miniforge3-MacOSX-arm64.sh\n\nconda create -n cnn-mac python=3.12\nconda activate cnn-mac\npip install tensorflow-macos tensorflow-metal numpy pandas opencv-python scikit-image scikit-learn\nimport tensorflow as tf\nprint("Num GPUs Available: ", len(tf.config.list_physical_devices(\'GPU\')))\n'

In [ ]:
# Import required libraries
import os
from skimage.io import imread, imsave
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
import gc
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.callbacks import EarlyStopping

print("Load environment successfully")

# Enable GPU memory growth for TensorFlow
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Create a new folder to store the flipped images
os.makedirs('train_ims_rever', exist_ok=True)

# Get all image names
image_names = os.listdir('train_ims')

# Flip images horizontally, add prefix and save them to the new folder
for img_name in image_names:
    img_path = os.path.join('train_ims', img_name)
    image = imread(img_path)
    image_reversed = np.fliplr(image)
    reversed_img_name = 'reverse_' + img_name
    reversed_img_path = os.path.join('train_ims_rever', reversed_img_name)
    imsave(reversed_img_path, image_reversed)

print("All images have been horizontally flipped and saved to 'train_ims_rever' folder with 'reverse_' prefix.")

# Read the original label file
df = pd.read_csv('train.csv')

# Create a new label file for the flipped images
df_reversed = df.copy()
df_reversed['im_name'] = 'reverse_' + df_reversed['im_name']

# Combine the original and flipped labels
df_combined = pd.concat([df, df_reversed], ignore_index=True)
df_combined.to_csv('train_combined.csv', index=False)

print("Labels for flipped images have been added to 'train_combined.csv'.")

# Read the combined label file
df = pd.read_csv('train_combined.csv')
labels = df['label'].values
image_names = df['im_name'].values

# Encode the labels (10 categories)
le = LabelEncoder()
labels = le.fit_transform(labels)

# Define image size and load images
IMG_SIZE = 128
channels = 3  # Grayscale; change to 3 if RGB
images = []
print("Start loading images...")
for img_name in image_names:
    if 'reverse_' in img_name:
        img_path = os.path.join('train_ims_rever', img_name)
    else:
        img_path = os.path.join('train_ims', img_name)
    image = cv2.imread(img_path)  # Grayscale
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    images.append(image)

images = np.array(images)
images = images.reshape(-1, IMG_SIZE, IMG_SIZE, channels) / 255.0  # Normalize
print("Images loaded, shape:", images.shape)

# Split data into 85% training and 15% testing
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.15, random_state=42)
print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)
del images, df, df_reversed, df_combined, image_names
gc.collect()



Load environment successfully
All images have been horizontally flipped and saved to 'train_ims_rever' folder with 'reverse_' prefix.
Labels for flipped images have been added to 'train_combined.csv'.
Start loading images...
Images loaded, shape: (100000, 128, 128, 3)
Training set shape: (85000, 128, 128, 3)
Testing set shape: (15000, 128, 128, 3)


35

In [3]:
# Data augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, channels)),
    data_augmentation,  # Data augmentation
    
    # Block 1 (32 filters)
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    # Block 2 (64 filters)
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    # Block 3 (128 filters)
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    # Block 4 (256 filters)
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    # Block 5 (512 filters)
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    # Block 6 (1024 filters)
    tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='valid'),  # No padding here
    tf.keras.layers.BatchNormalization(),
    
    # Global Average Pooling instead of Flatten (better for small feature maps)
    tf.keras.layers.GlobalAveragePooling2D(),
    
    # Dense layers
    tf.keras.layers.Dense(2048, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compile with a lower learning rate for better convergence
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model with more epochs
print("Training CNN model...")
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=45, batch_size=32, validation_split=0.1, callbacks=[early_stopping])

print("Model training completed")

# Evaluate on test set
print("Evaluating model on test set...")
test_predictions = model.predict(X_test)
test_predictions = np.argmax(test_predictions, axis=1)
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

2025-04-21 17:07:37.187640: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4 Pro
2025-04-21 17:07:37.188125: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 48.00 GB
2025-04-21 17:07:37.188131: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 18.00 GB
2025-04-21 17:07:37.188148: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-21 17:07:37.188163: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Training CNN model...
Epoch 1/45


2025-04-21 17:12:17.032157: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-04-21 17:12:17.040431: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.


2391/2391 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.4569 - loss: 1.5372

2025-04-21 17:15:40.606548: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.


2391/2391 ━━━━━━━━━━━━━━━━━━━━ 208s 86ms/step - accuracy: 0.4570 - loss: 1.5371 - val_accuracy: 0.6828 - val_loss: 0.8989
Epoch 2/45
2391/2391 ━━━━━━━━━━━━━━━━━━━━ 201s 84ms/step - accuracy: 0.6511 - loss: 0.9964 - val_accuracy: 0.7475 - val_loss: 0.7199
Epoch 3/45
2391/2391 ━━━━━━━━━━━━━━━━━━━━ 202s 84ms/step - accuracy: 0.7157 - loss: 0.8225 - val_accuracy: 0.7815 - val_loss: 0.6201
Epoch 4/45
2391/2391 ━━━━━━━━━━━━━━━━━━━━ 201s 84ms/step - accuracy: 0.7583 - loss: 0.6967 - val_accuracy: 0.8094 - val_loss: 0.5525
Epoch 5/45
2391/2391 ━━━━━━━━━━━━━━━━━━━━ 201s 84ms/step - accuracy: 0.7874 - loss: 0.6158 - val_accuracy: 0.8120 - val_loss: 0.5458
Epoch 6/45
2391/2391 ━━━━━━━━━━━━━━━━━━━━ 202s 84ms/step - accuracy: 0.8062 - loss: 0.5542 - val_accuracy: 0.8429 - val_loss: 0.4640
Epoch 7/45
2391/2391 ━━━━━━━━━━━━━━━━━━━━ 202s 84ms/step - accuracy: 0.8252 - loss: 0.5064 - val_accuracy: 0.8521 - val_loss: 0.4403
Epoch 8/45
2391/2391 ━━━━━━━━━━━━━━━━━━━━ 202s 84ms/step - accuracy: 0.8393 - lo

2025-04-21 19:43:57.230232: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.


469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step
Test Accuracy: 95.10%


In [5]:
model.save('cnn_model.keras')